In [1]:
import pandas as pd
import datetime
from tabulate import tabulate

In [2]:

try:
    df = pd.read_csv('user_database.csv')
    df.dropna(how="all", inplace=True)
    df.fillna('', inplace=True)
    df['password'] = df['password'].astype(str)
    user_db = df
except FileNotFoundError:
    user_db = {'username': ['admin'], 'password': [
        '111'], 'role': ['admin'], 'activated': ['TRUE'], 'email': ['hemtest11@gmail.com']}
    df = pd.DataFrame(user_db)
    #df.set_index('username', inplace=True)
    df['password'] = df['password'].astype(str)
    df.to_csv('user_database.csv',index=False)
    user_db = df
except:
    print("System couldn't read your user database file.")
    dataFailure = True

try:
    df = pd.read_csv('volunteer_database.csv')
    df.dropna(how="all", inplace=True)
    df.fillna('', inplace=True)
    vol_db = df
except FileNotFoundError:
    vol_db = {'Username': [''], 'First name': [''], 'Second name': [
        ''], 'Camp ID': [''], 'Avability': [''], 'Status': ['']}
    df = pd.DataFrame(vol_db)
    #df.set_index('Username', inplace=True)
    df.to_csv('volunteer_database.csv',index=False)
    vol_db = df
except:
    print("System couldn't read your volunteer database file.")
    dataFailure = True

try:
    df = pd.read_csv('refugee_database.csv')
    df.dropna(how="all", inplace=True)
    df.fillna('', inplace=True)
    refugee_db = df
except FileNotFoundError:
    refugee_db = {'Family ID': [''], 'Lead Family Member Name': [''], 'Lead Family Member Surname': [
        ''], 'Camp ID': [''], 'Mental State': [''], 'Physical State': [''], 'No. Of Family Members': ['']}
    df = pd.DataFrame(refugee_db)
    #df.set_index('Family ID', inplace=True)
    df.to_csv('refugee_database.csv',index=False)
    refugee_db = df
except:
    print("System couldn't read your refugees database file.")
    dataFailure = True

try:
    df = pd.read_csv('camp_database.csv')
    df.dropna(how="all", inplace=True)
    df.fillna('', inplace=True)
    camps_db = df
except FileNotFoundError:
    camps_db = {'Emergency ID': [''], 'Type of emergency': [''], 'Description': [''], 'Location': [''], 'Start date': [
        ''], 'Close date': [''], 'Number of refugees': [''], 'Camp ID': [''], 'No Of Volounteers': [''], 'Capacity': ['']}
    df = pd.DataFrame(camps_db)
    #df.set_index('Emergency ID', inplace=True)
    df.to_csv('camp_database.csv',index=False)
    camps_db = df
except:
    print("System couldn't read your camplist database file.")
    dataFailure = True

try:
    df = pd.read_csv('emergency_database.csv')
    df.dropna(how="all", inplace=True)
    df.fillna('', inplace=True)
    emergencies_db = df
except FileNotFoundError:
    emergencies_db = {'Emergency ID': [''], 'Location': [''], 'Type': [
        ''], 'Description': [''], 'Start date': [''], 'Close date': ['']}
    df = pd.DataFrame(emergencies_db)
    #df.set_index('Emergency ID', inplace=True)
    df.to_csv('emergency_database.csv',index=False)
    emergencies_db = df
except:
    print("System couldn't read your camplist database file.")
    dataFailure = True

try:
    df = pd.read_csv('mealplans_database.csv')
    df.dropna(how="all", inplace=True)
    df.fillna('', inplace=True)
    meals_db = df
except FileNotFoundError:
    meals_db = {'Camp ID': [''], 'Total number of refugees': [''],
               'Meals per day': [''], 'Days': [''], 'Total meals': [''],
               'Price per meal': [''], 'Budget per day': [''], 'Total budget': ['']}
    df = pd.DataFrame(meals_db)
    #df.set_index('Camp ID', inplace=True)
    df.to_csv('mealplans_database.csv',index=False)
    meals_db = df
except:
    print("System couldn't read the camps meals database file.")
    dataFailure = True


try:
    df = pd.read_csv(".sys_countries.csv", index_col='Country name')
    countries_db = df
except:
    print("System couldn't read the countries database file.")
    dataFailure = True

try:
    df = pd.read_csv('.sys_organisation_per_continent.csv')
    organisations_db = df
except:
    print("System couldn't read the Organisations per Continent database file.")
    dataFailure = True


In [5]:
'John' in vol_db['First name'].values and 'Green' in vol_db['Second name'].values

True

In [13]:
usrnm = vol_db.loc[vol_db['First name'] == 'John', 'Username'].values[0]

In [11]:
vol_db[['Username','Second name']]

,Username,Second name
0,Volunteer1,Green
1,Volunteer2,Smith
2,Volunteer3,Brown
3,Volunteer4,Richards
4,Volunteer5,Robinson
5,Volunteer6,Gold
6,Volunteer7,Vincent
7,Volunteer8,Vincent
8,Volunteer9,Thompson
9,Volunteer10,Jones


In [22]:
local_db = camps_db.merge(emergencies_db, on='Emergency ID', how='inner')
local_db_II = countries_db.reset_index().merge(organisations_db, on='Continent',how='left').set_index('Country name')
local_db_III = pd.merge(local_db_II, local_db, left_index=True, right_on='Location_x')
help = local_db_III.loc[local_db_III['Close date'] == '', ['Emergency ID', 'Camp ID', 'Type', 'Name of non-profit organisation', 'Email', 'Website']]
print(help)

  Emergency ID Camp ID      Type  \
3          PK1   PK1-1  Flooding   
4          PK1   PK1-2  Flooding   

                 Name of non-profit organisation           Email  \
3  Asia Pacific Alliance for Disaster Management  info@apadm.org   
4  Asia Pacific Alliance for Disaster Management  info@apadm.org   

             Website  
3  http://apadm.org/  
4  http://apadm.org/  


In [25]:
emergencies_db.loc[emergencies_db['Close date']=='']

,Emergency ID,Location,Type,Description,Start date,Close date
1,PK1,Pakistan,Flooding,Description,01/06/2022,
5,AM1,Armenia,Earthquake,Description,01/11/2022,
7,FJ1,Fiji,Earthquake,Description,01/07/2022,
8,GT1,Guatemala,Bushfire,Description,01/02/2022,
9,GT2,Guatemala,Flooding,Description,01/04/2022,
